In [1]:
# CAREFUL, DELETE LOG
import os, shutil

folder = '/tmp/tensorflow_logs/'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [2]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
import numpy as np
import pickle
from dataset_def import Dataset


In [3]:
def add_channel(data):
    return np.expand_dims(data, axis = data.ndim)

In [4]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean', mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var)

In [5]:
# Parameters
learning_rate = 0.001
training_iters = 20000
batch_size = 16
display_step = 10

logs_path = '/tmp/tensorflow_logs/shapes3D'

# Network Parameters
n_input = 28**3 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


In [6]:
# Create some wrappers for simplicity
def conv3d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv3d(x, W, strides=[1, strides, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool3d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool3d(x, ksize=[1, k, k, k, 1], strides=[1, k, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 28, 1])

    # Convolution Layer
    conv1 = conv3d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool3d(conv1, k=2)

    # Convolution Layer
    conv2 = conv3d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool3d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out



In [7]:
#layer_sizes = [32,64,1024]
layer_sizes = [8,16,256]
# Store layers weight & bias
weights = {
    # 5x5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 5, 1, layer_sizes[0]])),
    # 5x5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 5, layer_sizes[0], layer_sizes[1]])),
    # fully connected, 7*7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*7*layer_sizes[1], layer_sizes[2]])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([layer_sizes[2], n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([layer_sizes[0]])),
    'bc2': tf.Variable(tf.random_normal([layer_sizes[1]])),
    'bd1': tf.Variable(tf.random_normal([layer_sizes[2]])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}



In [8]:
# Load data
tf.logging.info('loading data...')
with open('data/shapes/3D_train_dataset.pickle', 'rb') as data:
    train_dataset = pickle.load(data)
    
with open('data/shapes/3D_test_dataset.pickle', 'rb') as data:
    test_dataset = pickle.load(data)
tf.logging.info('DONE')

INFO:tensorflow:loading data...
INFO:tensorflow:DONE


In [9]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)
#pred = inference(x)

# Define loss and optimizer
for weight in weights:
    regularizer += tf.nn.l2_loss(weight)
    
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    variable_summaries(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    variable_summaries(accuracy)


# Create a summary to monitor cost tensor
cost_summ = tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
acc_summ = tf.summary.scalar("accuracy", accuracy)
# Merge all summaries into a single op
merged = tf.summary.merge([cost_summ, acc_summ]);

# Initializing the variables
init = tf.global_variables_initializer()


In [11]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        
        batch_x, batch_y = train_dataset.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run([optimizer], feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            test_batch_x, test_batch_y = test_dataset.next_batch(256)
            accTest, summary = sess.run([accuracy, merged], feed_dict={x: test_batch_x,
                                      y: test_batch_y,
                                      keep_prob: 1.})
            summary_writer.add_summary(summary, step)
            print("Iter " + str(step*batch_size) + \
                  ", Loss= " + "{:.5f}".format(loss) + \
                  ", Training Accuracy= " + "{:.4f}".format(acc) + \
                  ", Testing Accuracy= " + "{:.4f}".format(accTest))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    test_batch_x, test_batch_y = test_dataset.next_batch(256)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_batch_x,
                                      y: test_batch_y,
                                      keep_prob: 1.}))
    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Iter 160, Loss= 13651.09766, Training Accuracy= 0.7500, Testing Accuracy= 0.6800
Iter 320, Loss= 10453.55078, Training Accuracy= 0.6250, Testing Accuracy= 0.7500
Iter 480, Loss= 8998.50977, Training Accuracy= 0.6875, Testing Accuracy= 0.7300
Iter 640, Loss= 1259.13184, Training Accuracy= 0.9375, Testing Accuracy= 0.7200
Iter 800, Loss= 2467.49292, Training Accuracy= 0.7500, Testing Accuracy= 0.7300
Iter 960, Loss= 4929.67822, Training Accuracy= 0.8125, Testing Accuracy= 0.7500
Iter 1120, Loss= 7385.10156, Training Accuracy= 0.8125, Testing Accuracy= 0.7400
Iter 1280, Loss= 2226.43555, Training Accuracy= 0.8125, Testing Accuracy= 0.7500
Iter 1440, Loss= 782.14038, Training Accuracy= 0.9375, Testing Accuracy= 0.7500
Iter 1600, Loss= 3910.49316, Training Accuracy= 0.8125, Testing Accuracy= 0.8100
Iter 1760, Loss= 10349.56250, Training Accuracy= 0.6250, Testing Accuracy= 0.8300
Iter 1920, Loss= 3572.85791, Training Accuracy= 0.8750, Testing Accuracy= 0.8000
Iter 2080, Loss= 4000.04834, Tra

KeyboardInterrupt: 

In [ ]:
# Second Structure

In [4]:
FC_SIZE = 1024
DTYPE = tf.float32


def _weight_variable(name, shape):
    return tf.get_variable(name, shape, DTYPE, tf.truncated_normal_initializer(stddev=0.1))


def _bias_variable(name, shape):
    return tf.get_variable(name, shape, DTYPE, tf.constant_initializer(0.1, dtype=DTYPE))

def inference(features):
     # Input Layer
    input_layer = tf.reshape(features, [-1, 28, 28, 28, 1])
    prev_layer = input_layer
    num_props = 1
    num_classes = 2
    
    in_filters = num_props
    with tf.variable_scope('conv1') as scope:
        out_filters = 16
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope.name)

        prev_layer = conv1
        in_filters = out_filters

    pool1 = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')
    norm1 = pool1  # tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta = 0.75, name='norm1')

    prev_layer = norm1

    with tf.variable_scope('conv2') as scope:
        out_filters = 32
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope.name)

        prev_layer = conv2
        in_filters = out_filters

    # normalize prev_layer here
    prev_layer = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

    with tf.variable_scope('conv3_1') as scope:
        out_filters = 64
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    with tf.variable_scope('conv3_2') as scope:
        out_filters = 64
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    with tf.variable_scope('conv3_3') as scope:
        out_filters = 32
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    # normalize prev_layer here
    prev_layer = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')


    with tf.variable_scope('local3') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        prev_layer_flat = tf.reshape(prev_layer, [-1, dim])
        weights = _weight_variable('weights', [dim, FC_SIZE])
        biases = _bias_variable('biases', [FC_SIZE])
        local3 = tf.nn.relu(tf.matmul(prev_layer_flat, weights) + biases, name=scope.name)

    prev_layer = local3

    with tf.variable_scope('local4') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        prev_layer_flat = tf.reshape(prev_layer, [-1, dim])
        weights = _weight_variable('weights', [dim, FC_SIZE])
        biases = _bias_variable('biases', [FC_SIZE])
        local4 = tf.nn.relu(tf.matmul(prev_layer_flat, weights) + biases, name=scope.name)

    prev_layer = local4

    with tf.variable_scope('softmax_linear') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        weights = _weight_variable('weights', [dim, num_classes])
        biases = _bias_variable('biases', [num_classes])
        softmax_linear = tf.add(tf.matmul(prev_layer, weights), biases, name=scope.name)

    return softmax_linear

def cnn_model_fn(features, labels, mode):
  #"""Model function for CNN."""
  # Input Layer

  # Logits Layer
    logits = inference(features)
    

    loss = None
    train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.001,
            optimizer="SGD")

  # Generate Predictions
    predictions = {
        "classes": tf.argmax(
          input=logits, axis=1),
        "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

  # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
        mode=mode, predictions=predictions, loss=loss, train_op=train_op)


In [ ]:
# Construct model
#pred = conv_net(x, weights, biases, keep_prob)
pred = inference(x)

# Define loss and optimizer
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    variable_summaries(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    variable_summaries(accuracy)


# Create a summary to monitor cost tensor
cost_summ = tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
acc_summ = tf.summary.scalar("accuracy", accuracy)
# Merge all summaries into a single op
merged = tf.summary.merge([cost_summ, acc_summ]);

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        
        batch_x, batch_y = train_dataset.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run([optimizer], feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            test_batch_x, test_batch_y = test_dataset.next_batch(256)
            accTest, summary = sess.run([accuracy, merged], feed_dict={x: test_batch_x,
                                      y: test_batch_y,
                                      keep_prob: 1.})
            summary_writer.add_summary(summary, step)
            print("Iter " + str(step*batch_size) + \
                  ", Loss= " + "{:.5f}".format(loss) + \
                  ", Training Accuracy= " + "{:.4f}".format(acc) + \
                  ", Testing Accuracy= " + "{:.4f}".format(accTest))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    test_batch_x, test_batch_y = test_dataset.next_batch(256)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_batch_x,
                                      y: test_batch_y,
                                      keep_prob: 1.}))
    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")